In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo

## Data

In [2]:
gen = pd.DataFrame({
                    'id':      [1, 2, 3, 4],
                    'node':    [1, 1, 2, 3],
                    'max_gen': [140, 285, 90, 85],
                    'm_cost':  [7.5, 6.0, 14.0, 10.0], 
})

line = pd.DataFrame({
                    'id':   [1, 2, 3],
                    'from': [1, 1, 2],
                    'to':   [2, 3, 3],
                    'cap':  [126, 250, 130], 
                    'B':    [1/0.2, 1/0.2, 1/0.1],
})

bus = pd.DataFrame({
                    'id': [1, 2, 3],
                    'load': [50, 60, 300],
})

print(f'gen')
display(gen)
print(f'line')
display(line)
print(f'bus')
display(bus)

gen


,id,node,max_gen,m_cost
0,1,1,140,7.5
1,2,1,285,6.0
2,3,2,90,14.0
3,4,3,85,10.0


line


,id,from,to,cap,B
0,1,1,2,126,5.0
1,2,1,3,250,5.0
2,3,2,3,130,10.0


bus


,id,load
0,1,50
1,2,60
2,3,300


In [3]:
def read_data(bus, gen, line):
    bus = bus.set_index('id')
    gen = gen.set_index('id')
    line = line.set_index('id')

    return bus, gen, line

## Operation

In [4]:
def DCOPF(Bus, Gen, Line):
    model = pyo.ConcreteModel()
    N=Bus.index
    G=Gen.index
    K=Line.index
    
    # Define Variables
    model.p = pyo.Var(G, bounds=(0, None)) # power는 nonnegative
    model.f = pyo.Var(K, bounds=(0, None))
    model.theta = pyo.Var(N)
    
    # Objective Function
    #model.obj = pyo.Objective(expr=sum(model.p[g]*Gen.loc[g, 'vCost1'] for g in G))
    model.obj = pyo.Objective(expr=sum(model.p[g]*Gen.loc[g, 'm_cost'] for g in G))
    
    # Constraints
    model.gen_max = pyo.ConstraintList()
    model.flow = pyo.ConstraintList()
    model.flow_max = pyo.ConstraintList()
    model.node_balance = pyo.ConstraintList()
    model.slack = pyo.ConstraintList()
    
    # 최대 출력 제약.
    for g in G:
        model.gen_max.add(model.p[g] <= Gen.loc[g, 'max_gen'])
    
    for k in K:
        model.flow.add(model.f[k] == Line.loc[k,'B'] *
                       (model.theta[Line.loc[k,'from']]-model.theta[Line.loc[k,'to']]))
        
        model.flow_max.add(model.f[k] <= Line.loc[k, 'cap'])


    # Slack의 theta는 0으로 고정.
    model.slack.add(model.theta[3]==0)
    
    for n in N:
        demand = Bus.loc[n,'load']
        inflow = 0
        outflow = 0
        localgen = 0

        for k in K:
            if Line.loc[k,'to']==n:
                inflow += model.f[k]
            if Line.loc[k,'from']==n:
                outflow += model.f[k]

        for g in G:
            if Gen.loc[g,'node']==n:
                localgen += model.p[g]
        model.node_balance.add(localgen+inflow-outflow==demand)
        
    return model

In [5]:
# Read Data
#print(os.getcwd())
Bus, Gen, Line = read_data(bus, gen, line)
print("Data was read successfully.")
N=Bus.index
G=Gen.index
K=Line.index

# Construct Model
model = DCOPF(Bus, Gen, Line)
print("Construction of model complete.")

# Identify the solver
opt = pyo.SolverFactory('gurobi')

# Tell the model that you need the dual values too!
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Solve the problem
Solution=opt.solve(model)

# Report the summary of solve
Solution.write(num=1)

# Report the output
for g in G:
    print('Generator ',g,'Output (MW):', model.p[g].value)
    
for k in K:
    print('Line ',k,'Flow (MW):', model.f[k].value)

for n in N:
    print('LMP for node ', n, 'in ($/MWh) is: ', model.dual[model.node_balance[n]])

'''
for k in K:
    print('Flowgate price for line', k, 'in ($/MWh) in both directions are: ',
            model.dual[model.flow_max[2*k-1]],' and ', model.dual[model.flow_max[2*k]])
'''

Data was read successfully.
Construction of model complete.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x11
  Lower bound: 2835.0
  Upper bound: 2835.0
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 11
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 11
  Number of nonzeros: 28
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optima

"\nfor k in K:\n    print('Flowgate price for line', k, 'in ($/MWh) in both directions are: ',\n            model.dual[model.flow_max[2*k-1]],' and ', model.dual[model.flow_max[2*k]])\n"